In [16]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.ByteString as B
import qualified Data.ByteString.Char8 as B (lines)


input <- tail . tail . lines <$> readFile "../input/day22.txt"

words $ head input

parse line = let
    [identity, cap, used, avail, _] = words line
    in (identity, read $ init used :: Int, read $ init avail :: Int, read $ init cap :: Int)

parsed = map parse input
head parsed

["/dev/grid/node-x0-y0","87T","71T","16T","81%"]

("/dev/grid/node-x0-y0",71,16,87)

In [17]:
viable (name,used,_,_) list = length $ filter (\(n,_,avail,_) -> n /= name && avail >= used && used > 0) list

counts = map (flip viable parsed) parsed
sum counts

892